ETL [Extract -- Transform -- Load]

In [ ]:
import pandas as pd

# Extract
customers = pd.read_csv('/content/g_dim_customers.csv', encoding='latin1')
products = pd.read_csv('/content/g_dim_products.csv', encoding='latin1')
sales = pd.read_csv('/content/g_fact_sales.csv', encoding='latin1')


print("✅ Files loaded")

# Inspect and Clean (Transform)
print("\n👉 Customers sample:")
print(customers.head())

print("\n👉 Products sample:")
print(products.head())

print("\n👉 Sales sample:")
print(sales.head())

# Check for missing data
print("\nMissing values in Customers:")
print(customers.isnull().sum())

print("\nMissing values in Products:")
print(products.isnull().sum())

print("\nMissing values in Sales:")
print(sales.isnull().sum())

# Example cleaning:
# Remove duplicates
customers = customers.drop_duplicates()
products = products.drop_duplicates()
sales = sales.drop_duplicates()

# Convert dates if present
if 'order_date' in sales.columns:
    sales['order_date'] = pd.to_datetime(sales['order_date'], errors='coerce')
if 'ship_date' in sales.columns:
    sales['ship_date'] = pd.to_datetime(sales['ship_date'], errors='coerce')

# Example derived feature: shipping days
if 'order_date' in sales.columns and 'ship_date' in sales.columns:
    sales['shipping_days'] = (sales['ship_date'] - sales['order_date']).dt.days




✅ Files loaded

👉 Customers sample:
   customer_key  customer_id customer_number first_name last_name    country  \
0             1        11000      AW00011000        Jon      Yang  Australia   
1             2        11001      AW00011001     Eugene     Huang  Australia   
2             3        11002      AW00011002      Ruben    Torres  Australia   
3             4        11003      AW00011003    Christy       Zhu  Australia   
4             5        11004      AW00011004  Elizabeth   Johnson  Australia   

   gender marital_status  birth_date  create_date  
0    Male        Married  1971-10-06  10/6/2025\r  
1    Male         Single  1976-05-10  10/6/2025\r  
2    Male        Married  1971-02-09  10/6/2025\r  
3  Female         Single  1973-08-14  10/6/2025\r  
4  Female         Single  1979-08-05  10/6/2025\r  

👉 Products sample:
   product_key  product_id product_number               product_name  \
0            1         210     FR-R92B-58  HL Road Frame - Black- 58   
1      

In [ ]:
#know about columns in dataset
print("Sales columns:", sales.columns.tolist())
print("Customers columns:", customers.columns.tolist())
print("Products columns:", products.columns.tolist())


Sales columns: ['order_number', 'customer_key', 'product_key', 'order_date', 'shipping_date', 'due_date', 'sales', 'quantity', 'price']
Customers columns: ['customer_key', 'customer_id', 'customer_number', 'first_name', 'last_name', 'country', 'gender', 'marital_status', 'birth_date', 'create_date']
Products columns: ['product_key', 'product_id', 'product_number', 'product_name', 'category_id', 'category', 'sub_category', 'maintenance', 'product_ine', 'cost', 'start_date']


In [ ]:
# Merge
df = sales.merge(customers, on='customer_key', how='left') \
          .merge(products, on='product_key', how='left')

print("\n✅ Final merged dataset shape:", df.shape)

# Check missing values
print("\nMissing values in final dataset:")
print(df.isnull().sum())

# Fill numeric NaNs with 0
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    df[col] = df[col].fillna(0)

# Fill object NaNs with 'Unknown'
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].fillna('Unknown')

# Sample
print("\n👉 Cleaned data sample:")
print(df.head())

# Export
df.to_csv('cleaned_sales_data.csv', index=False)
print("\n✅ Cleaned dataset exported to: cleaned_sales_data.csv")



✅ Final merged dataset shape: (60398, 28)

Missing values in final dataset:
order_number         0
customer_key         0
product_key          0
order_date          19
shipping_date        0
due_date             0
sales                0
quantity             0
price                0
customer_id          0
customer_number      0
first_name           6
last_name            6
country            871
gender              33
marital_status       4
birth_date          78
create_date          0
product_id           0
product_number       0
product_name         0
category_id          0
category             0
sub_category         0
maintenance          0
product_ine          0
cost                 0
start_date           0
dtype: int64

👉 Cleaned data sample:
  order_number  customer_key  product_key order_date shipping_date  \
0      SO43697         10769           20 2010-12-29    2011-01-05   
1      SO43698         17390            9 2010-12-29    2011-01-05   
2      SO43699         14864    